In [2]:
import json
import numpy as np
import cv2

path = 'ClimbingHoldDetection-15/train'

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [36]:
with open('ClimbingHoldDetection-15/train/_annotations.coco.json') as f:
    file = json.loads(f.read())
    images = file['images']
    annotations = file['annotations']
    

In [37]:
annotations[1]

{'id': 1,
 'image_id': 0,
 'category_id': 5,
 'bbox': [202, 104, 137.5, 70.5],
 'area': 9693.75,
 'segmentation': [],
 'iscrowd': 0}

In [44]:
images[0]

{'id': 0,
 'license': 1,
 'file_name': '20231009_133118_jpg.rf.a4403489a7a5e6fb3150293fb413dd6f.jpg',
 'height': 640,
 'width': 640,
 'date_captured': '2023-11-06T01:50:53+00:00'}

In [50]:
len(images)

382

In [38]:
def extract_bbox(image, bbox_coord):
    x, y, w, h = [int(b) for b in bbox_coord]
    return image[y:y+h, x:x+w]


def parse_annotations(annotations):
    img_id_to_annotations = {}

    for a in annotations:
        if a['image_id'] in img_id_to_annotations:
            img_id_to_annotations[a['image_id']].append(a['id'])
        else:
            img_id_to_annotations[a['image_id']] = [a['id']]

    return img_id_to_annotations

In [39]:
img_id_to_annotations = parse_annotations(annotations)

In [45]:
import os
def getBoudingBoxForImage(imageId, img_id_to_annotations, annotations, images, path = 'ClimbingHoldDetection-15/train', saving_dir=None):
    annotation_ids = img_id_to_annotations[imageId]
    img_path =  os.path.join(path, images[imageId]['file_name'])

    image = cv2.imread(img_path)


    for a_id in annotation_ids:
        extracted_img = extract_bbox(image, annotations[a_id]['bbox'])
        if saving_dir:
            cv2.imwrite(os.path.join(saving_dir, f"{imageId:05d}_{a_id:05d}.png"), extracted_img)

In [46]:
getBoudingBoxForImage(0, img_id_to_annotations, annotations, images, saving_dir="extractedLabeledDataset")

In [48]:
def extractAllImages(interval, img_id_to_annotations, annotations, images, path = 'ClimbingHoldDetection-15/train', saving_dir=None):
    from tqdm import tqdm
    for i in tqdm(interval):
        getBoudingBoxForImage(i, img_id_to_annotations, annotations, images, saving_dir="extractedLabeledDataset")

In [49]:
extractAllImages(range(0,10), img_id_to_annotations, annotations, images, saving_dir="extractedLabeledDataset")

100%|██████████| 10/10 [00:00<00:00, 109.70it/s]


In [73]:
desired_class = 4
count = 0
i = 0 
while count < 10:
    if annotations[i]['category_id'] == desired_class:
        print(images[annotations[i]['image_id']])
        print(annotations[i]['id'])
        count += 1

    i+= 1

{'id': 2, 'license': 1, 'file_name': '20231009_133346_jpg.rf.a080d11ad88cfdb6f079a2c85162d447.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-11-06T01:50:53+00:00'}
66
{'id': 2, 'license': 1, 'file_name': '20231009_133346_jpg.rf.a080d11ad88cfdb6f079a2c85162d447.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-11-06T01:50:53+00:00'}
67
{'id': 2, 'license': 1, 'file_name': '20231009_133346_jpg.rf.a080d11ad88cfdb6f079a2c85162d447.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-11-06T01:50:53+00:00'}
75
{'id': 5, 'license': 1, 'file_name': '20231009_133347_jpg.rf.989af4e60cffc56f5f6cc29b5a40aaff.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-11-06T01:50:53+00:00'}
144
{'id': 5, 'license': 1, 'file_name': '20231009_133347_jpg.rf.989af4e60cffc56f5f6cc29b5a40aaff.jpg', 'height': 640, 'width': 640, 'date_captured': '2023-11-06T01:50:53+00:00'}
146
{'id': 5, 'license': 1, 'file_name': '20231009_133347_jpg.rf.989af4e60cffc56f5f6cc29b5a40aaff.jpg', 'height'

In [ ]:
# 1: Crimp
# 2: Jug
# 3: Pinch
# 4: Pocket
# 5: Sloper